# Ch08. OECD 국가의 GDP데이터로 실무의 데이터 다루기

## 1. 결측값 채우기 (fillna)

### 학습 목표
- 결측값을 지정한 값으로 채운다. (`fillna` 함수의 `value` 인자)
- 결측값을 근접한 값으로 채운다. (`fillna` 함수의 `method` 인자)

> pandas fillna


결측값 null(대표적으로 NaN)을 지정한 값으로 채우는 함수


<img src=https://i.ibb.co/rcbvvQ5/08-01.jpg, width=600>


**value** (scalar, 딕셔너리, 시리즈, 데이터 프레임)

NaN을 채울 값

- scalar를 입력하면 동일한 값으로 채운다

- 딕셔너리나 시리즈를 입력하면 key에 맞는 열마다 다른 값을 채운다

- 데이터 프레임은 인덱스가 동일한 값을 채운다
 

**method** (인수는 ‘backfill’, ‘bfill’, ‘pad’, ‘ffill’, None / 기본값은 None)

근접한 값으로 NaN을 채울 때 사용하는 인자

- ffill / pad : 이전 값으로 채운다
- bfill / backfill: 이후의 값으로 채운다

 
<img src=https://i.ibb.co/9h9cshK/08-01-02.jpg, width=600>
 

 

**axis** (인수는 0 또는 1 / 기본값은 0)

method로 근접한 값을 채울 때 축을 지정하는 인자, 0이면 열에서 근접한 값을 채운다

<br><br>
 

fillna 공식문서 : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html



In [1]:
# 실습 준비 코드
import pandas as pd
data = [[88, 66, None], [None, None, 69], [69, 82, None], [71, 89, 98]]
data1 = [[0, 1, 2] , [3, 4, 5], [6, 7, 8], [9, 10, 11]]
df = pd.DataFrame(data, index=list('ABCD'), columns=['국어', '영어', '수학'])
df1 = pd.DataFrame(data1, index=df.index, columns=df.columns) 
df

,국어,영어,수학
A,88.0,66.0,NaN
B,NaN,NaN,69.0
C,69.0,82.0,NaN
D,71.0,89.0,98.0


In [2]:
# 값으로 채우기 (scalar로 채우기)
df.fillna(0)

,국어,영어,수학
A,88.0,66.0,0.0
B,0.0,0.0,69.0
C,69.0,82.0,0.0
D,71.0,89.0,98.0


In [3]:
# 맵퍼로 채우기 (열마다 다른 값으로 채운다)
df.fillna({'국어': 0, '영어': 1, '수학': 2})

,국어,영어,수학
A,88.0,66.0,2.0
B,0.0,1.0,69.0
C,69.0,82.0,2.0
D,71.0,89.0,98.0


In [4]:
# 모든 열을 key에 지정할 필요는 없다
df.fillna({'국어': 0, '영어': 1})

,국어,영어,수학
A,88.0,66.0,NaN
B,0.0,1.0,69.0
C,69.0,82.0,NaN
D,71.0,89.0,98.0


In [5]:
# 열의 평균으로 채우기
df.fillna(df.mean())

,국어,영어,수학
A,88.0,66.0,83.5
B,76.0,79.0,69.0
C,69.0,82.0,83.5
D,71.0,89.0,98.0


In [6]:
# 행의 평균으로 채우는 것은 기본으로는 불가능하니 lambda 함수를 만들자
x = df.loc['A']
x.fillna(x.mean())

국어    88.0
영어    66.0
수학    77.0
Name: A, dtype: float64

In [7]:
# 시리즈의 평균값으로 시리즈를 채우는 lambda 함수를 apply로 행별(axis=1)로 적용
df.apply(lambda x: x.fillna(x.mean()), axis=1)

,국어,영어,수학
A,88.0,66.0,77.0
B,69.0,69.0,69.0
C,69.0,82.0,75.5
D,71.0,89.0,98.0


In [8]:
# 열의 전방값으로 채우기
df.fillna(method='ffill')

,국어,영어,수학
A,88.0,66.0,NaN
B,88.0,66.0,69.0
C,69.0,82.0,69.0
D,71.0,89.0,98.0


In [9]:
# 열의 전방값으로 채우는 함수 ffill
df.ffill()

,국어,영어,수학
A,88.0,66.0,NaN
B,88.0,66.0,69.0
C,69.0,82.0,69.0
D,71.0,89.0,98.0


In [10]:
# 열의 후방값으로 채우기
df.fillna(method='bfill')

,국어,영어,수학
A,88.0,66.0,69.0
B,69.0,82.0,69.0
C,69.0,82.0,98.0
D,71.0,89.0,98.0


In [11]:
# 열의 후방값으로 채우는 함수 bfill
df.bfill()

,국어,영어,수학
A,88.0,66.0,69.0
B,69.0,82.0,69.0
C,69.0,82.0,98.0
D,71.0,89.0,98.0


In [12]:
# 행의 전방값으로 채우기 (axis=1)
df.fillna(method='ffill', axis=1)

,국어,영어,수학
A,88.0,66.0,66.0
B,NaN,NaN,69.0
C,69.0,82.0,82.0
D,71.0,89.0,98.0


## 2. 열을 레이블로 필터링하기(filter)

### 학습 목표
- 열을 이름에 문자열 포함 여부로 필터링한다. (`filter` 함수의 `like` 인자)
- 열의 이름을 간단한 정규 표현식으로 필터링한다. (`filter` 함수의 `regex` 인자)

> pandas filter


인덱스의 레이블로 데이터프레임이나 시리즈를 필터링하는 함수

<img src=https://i.ibb.co/hXvVHKW/08-02-01.jpg, width=600>



**items**

필터링때 정확하게 일치하는 문자열을  지정하는 인자

 

**like**

필터링때 포함하는 문자열을  지정하는 인자

 

**regex** (regular expression)

필터링때 정규 표현식을 지정하는 인자

 

**axis** (0 or 1 / 기본값은 1)

index를 필터링하는지 columns를 필터링하는지 결정하는 인자

<br><br>

filter 판다스 공식문서 : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.filter.html

In [13]:
# 실습 준비 코드
import pandas as pd
data = [[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11]]
df = pd.DataFrame(data, columns=['A', 'BC', 'AC', 'DA'])
df

,A,BC,AC,DA
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [14]:
# 레이블에 A가 포함되는 열 필터링
df.filter(like='A')

,A,AC,DA
0,0,2,3
1,4,6,7
2,8,10,11


In [15]:
# 레이블이 A로 시작되는 열 필터링
df.filter(regex=r'^A')

,A,AC
0,0,2
1,4,6
2,8,10


In [16]:
# 레이블이 B 또는 C를 포함하는 열 필터링
df.filter(regex=r'B|C')

,BC,AC
0,1,2
1,5,6
2,9,10


정규표현식을 배우지 않은 사람들도

- ^ : 시작
- $ : 끝 
- | : 또는

간단한 위 세가지만 활용해도 크게 도움이 된다

## 3. 실무 데이터의 문제점


OECD 국가 주요 지표 : https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_2KAAG01

**문제점**
1. columns가 멀티인덱스인데도 열이 너무 많다. 이런 피벗테이블은 한눈에 알아볼 수 없다

2. 국가의 데이터에 대륙을 구분하기 위한 행이 존재한다

3. 각 대륙의 GDP가 높은 나라는?

  <img src=https://i.ibb.co/KXG9qWQ/08-03-01.png, width=300>


→ 피벗테이블로서의 기능도 못하고 데이터를 집계할수 있도록 관리하는 기능도 못하고 있다


**raw 데이터 처럼 집계를 할 수 있는 데이터로 만들어 필요한 요약은 피벗테이블로 집계해 제공할수 있으면 문제가 해결된다**

## 4. OECD의 GDP 데이터 전처리하기

통계청 데이터는 전처리해야 될것이 지나치게 많아 약간의 전처리를 한 데이터를 사용하겠습니다

In [17]:
# 프로젝트 코드
import pandas as pd
pd.options.display.max_rows = 6
url = './08_01_oecd_gdp.csv'

<img src=https://i.ibb.co/6r5G71s/1.jpg, width=600>

In [18]:
df_ex1 = pd.read_csv(url, header=[0, 1], index_col=0)
df_ex1

2019                                       2020                  \
         1인당 GDP GDP 성장률     GDP      수입        수출  1인당 GDP GDP 성장률     GDP   
국가                                                                            
아시아          NaN     NaN     NaN     NaN       NaN      NaN     NaN     NaN   
대한민국     31929.0     2.2  1651.0  503343  542233.0  31637.0    -0.9  1638.2   
이스라엘     43589.0     3.5   394.7   75697   51899.0  43611.0    -2.4   402.0   
...          ...     ...     ...     ...       ...      ...     ...     ...   
오세아니아        NaN     NaN     NaN     NaN       NaN      NaN     NaN     NaN   
오스트레일리아  55057.0     2.2  1396.6  154043  271006.0  51812.0    -0.3  1330.9   
뉴질랜드     41999.0     1.6   209.1   42362   39516.0  41792.0     1.0   212.5   

                         
             수입      수출  
국가                       
아시아         NaN     NaN  
대한민국     467633  512498  
이스라엘      69810   49372  
...         ...     ...  
오세아니아       NaN     NaN  
오스트레일리아  146522  250570  
뉴질랜드      37135   38897  

[43 rows x 10 columns]

<img src=https://i.ibb.co/FXmmzv7/2.jpg, width=600>

In [19]:
# 필요한 GDP열만 남긴 뒤 연도와 국가를 열로 만들고 열 이름을 수정
df_ex2 = (df_ex1
          .stack(0, dropna=False) #피봇테이블을 누적데이터로 변환
          .filter(like='GDP')
          .reset_index() #인덱스 정리
          .rename(columns={'level_1':'연도'}))
df_ex2

,국가,연도,1인당 GDP,GDP,GDP 성장률
0,아시아,2019,NaN,NaN,NaN
1,아시아,2020,NaN,NaN,NaN
2,대한민국,2019,31929.0,1651.0,2.2
...,...,...,...,...,...
83,오스트레일리아,2020,51812.0,1330.9,-0.3
84,뉴질랜드,2019,41999.0,209.1,1.6
85,뉴질랜드,2020,41792.0,212.5,1.0


<img src=https://i.ibb.co/41RCL6R/3.jpg, width=600>

In [20]:
# 대륙구분을 행에서 하는 것이 아니고 구분 열을 만들고 대륙 구분 행 삭제
cond1 = df_ex2['GDP'].isnull()
df_ex2['구분'] = df_ex2['국가'].mask(~cond1).fillna(method='ffill')
df_ex3 = df_ex2[~cond1].reset_index(drop=True)
df_ex3

,국가,연도,1인당 GDP,GDP,GDP 성장률,구분
0,대한민국,2019,31929.0,1651.0,2.2,아시아
1,대한민국,2020,31637.0,1638.2,-0.9,아시아
2,이스라엘,2019,43589.0,394.7,3.5,아시아
...,...,...,...,...,...,...
73,오스트레일리아,2020,51812.0,1330.9,-0.3,오세아니아
74,뉴질랜드,2019,41999.0,209.1,1.6,오세아니아
75,뉴질랜드,2020,41792.0,212.5,1.0,오세아니아


**df_ex3는 집계가 가능한 raw data와 마찬가지이므로 다양한 집계를 해보자**

In [21]:
# 연도별 대륙별 GDP 성장률의 최대 피벗 테이블
df_ex3.pivot_table('GDP 성장률', index='구분', columns='연도', 
                   aggfunc='max')

연도,2019,2020
구분,,
남아메리카,3.3,-4.5
북아메리카,2.2,-3.5
아시아,3.5,1.8
오세아니아,2.2,1.0
유럽,5.6,3.4


In [22]:
# 연도별 대륙별 GDP 성장률의 최대인 국가 피벗 테이블
df_ex3.pivot_table('GDP 성장률', index='구분', columns='연도', 
                   aggfunc=lambda x: df_ex3.loc[x.idxmax(), '국가'])

연도,2019,2020
구분,,
남아메리카,콜롬비아,코스타리카
북아메리카,미국,미국
아시아,이스라엘,튀르키예
오세아니아,오스트레일리아,뉴질랜드
유럽,아일랜드,아일랜드
